# How to use AnySDK to make a dynamic toolkit

AnySDK allows you to wrap any SDK client that exposes various CRUD operations (get, update, create, delete) and turn them into LLM-enabled toolkits without having to write a custom toolkit.

There are two core aspects to AnySDK:
1. AnySDKWrapper -- wraps the SDK you pass in
2. CrudControls -- allows you to control the access an agent has to Create, Read, Update, Delete verbs.

As of writing, using Anthropic's Claude3 Sonnet model in this notebook will cost less than USD 0.50 to complete.

First, we'll create a client SDK. Let's use AWS's `boto3` library. Ensure you have your credentials available or set them here. 

In [1]:
import getpass
import os

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass(prompt="AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass(prompt="AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Let's validate that your S3 client is able to make an API call. This `list_buckets` command should match how many buckets are in your account.

In [2]:
import boto3

s3 = boto3.client("s3")
buckets = s3.list_buckets()
print(len(buckets["Buckets"]))

9


Now, we can pass this client into AnySDKWrapper to create the toolkit. The callable functions within the SDK will become available as tools within the `anysdk.operations` object.

_NOTE_ If your SDK has many callable functions, your tool list could exceed your model's context length. Use CrudControls to limit the tools your Agent has access to.

To demonstrate this, we're limiting the `read_list` parameter to a single function, `list_buckets`. You can add more functions by using a prefix (ie, `list` or multiple values by using a comma separated list: `get,read,list`)

In [3]:
from langchain_community.utilities.anysdk import AnySdkWrapper, CrudControls

client = {"client": s3}
crud_controls = CrudControls(
    read_list="list_buckets",
)

anysdk = AnySdkWrapper(client=client, crud_controls=crud_controls)

print([tool.name for tool in anysdk.operations])

['list_buckets']


Since `anysdk.operations` is a list of tools, we can pass that into an Agent upon initialization and use them.

In [4]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

==============

In [5]:
# Now we'll set the Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass(prompt="Anthropic API Key: ")

With your API key set, we'll be able to create the AgentExecutor.

In [6]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

agent = create_structured_chat_agent(llm, anysdk.operations, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=anysdk.operations,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,
)

Now we can use this agent to perform functions for us using the configured AWS `boto3` client! Let's ask the Agent how many buckets we have.

In [7]:
agent_executor.invoke(
    {
        "input": "How many S3 buckets do I have? Pass an empty string as the 'action_input'."
    }
)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "list_buckets",
  "action_input": ""
}
```
{"ResponseMetadata": {"RequestId": "QRQCCSP8W2CCF86G", "HostId": "5ZBd6lyHgpqJ4eQMYUHwFN/0lKvpJi516pNiyOQBqSPI/XvelezWtjZ9UcCxUra5xj4ake04Ulc=", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amz-id-2": "5ZBd6lyHgpqJ4eQMYUHwFN/0lKvpJi516pNiyOQBqSPI/XvelezWtjZ9UcCxUra5xj4ake04Ulc=", "x-amz-request-id": "QRQCCSP8W2CCF86G", "date": "Tue, 14 May 2024 15:20:51 GMT", "content-type": "application/xml", "transfer-encoding": "chunked", "server": "AmazonS3"}, "RetryAttempts": 0}, "Buckets": [{"Name": "amplify-carboncounting-dev-100223-deployment", "CreationDate": "2019-10-26 14:02:32+00:00"}, {"Name": "casestudypro-api-dev-serverlessdeploymentbucket-1k1jafqbcjdzq", "CreationDate": "2022-02-19 19:12:41+00:00"}, {"Name": "casestudypro-api-prod-serverlessdeploymentbucket-12tvawyzsxx2l", "CreationDate": "2021-10-27 15:41:09+00:00"}, {"Name": "casestudypro-prod", "CreationDate": "2021-10-27 15:

{'input': "How many S3 buckets do I have? Pass an empty string as the 'action_input'.",
 'output': 'Based on the output from the list_buckets tool, you have 9 S3 buckets.'}

Now, let's have our Agent create a bucket. We'll need to modify the CrudControls to include the `create_bucket` function from the `boto3` SDK.

We'll also need to enable the create actions as well by setting `create` to `True`.

_WARNING_ This will create a bucket in your account! (Buckets are free to create, so long as you don't store data in them.)

In [8]:
from langchain_community.utilities.anysdk import AnySdkWrapper, CrudControls

client = {"client": s3}
crud_controls = CrudControls(
    read_list="list_buckets", create=True, create_list="create_bucket"
)

anysdk = AnySdkWrapper(client=client, crud_controls=crud_controls)

print([tool.name for tool in anysdk.operations])

['create_bucket', 'list_buckets']


More complicated tools often require more complex logic which requires more capable models. To create a bucket, we'll use the Sonnet model available from Anthropic.

In [9]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

agent = create_structured_chat_agent(llm, anysdk.operations, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=anysdk.operations,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,
)

In [10]:
import random

rand = str(random.randint(0, 10000000))
agent_executor.invoke(
    {"input": f"Create a bucket named 'my-test-bucket-{rand}' in my account."}
)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "create_bucket",
  "action_input": {
    "Bucket": "my-test-bucket-5246662"
  }
}
```
{"ResponseMetadata": {"RequestId": "J5S30XDPWG0EH4WF", "HostId": "3t1u5xb73plbtNyEXHKqy5LHSkXg/eKN9smbSpi52hiIbm5OE0eBZzsR/vpLtEhUoTp3i3eH3m+yAi38hknp48LPzb59u0VFiWB9niPJDak=", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amz-id-2": "3t1u5xb73plbtNyEXHKqy5LHSkXg/eKN9smbSpi52hiIbm5OE0eBZzsR/vpLtEhUoTp3i3eH3m+yAi38hknp48LPzb59u0VFiWB9niPJDak=", "x-amz-request-id": "J5S30XDPWG0EH4WF", "date": "Tue, 14 May 2024 15:22:12 GMT", "location": "/my-test-bucket-5246662", "server": "AmazonS3", "content-length": "0"}, "RetryAttempts": 0}, "Location": "/my-test-bucket-5246662"}Action:
```json
{
  "action": "Final Answer",
  "action_input": "The bucket 'my-test-bucket-5246662' was successfully created in your AWS account."
}
```

> Finished chain.


{'input': "Create a bucket named 'my-test-bucket-5246662' in my account.",
 'output': "The bucket 'my-test-bucket-5246662' was successfully created in your AWS account."}

To clean up the bucket you just created, run the below code block. Of course, you could also clean up with the Agent by modifying the CrudControls accordingly. 

In [11]:
s3.delete_bucket(Bucket=f"my-test-bucket-{rand}")

{'ResponseMetadata': {'RequestId': 'N3T5XEASVGEB4R5T',
  'HostId': 'gvN7+feJ9DIIMXUudl6jdQFdPkDX7iTYc4uVI038Jn+PRXy4A/zgTpnXhp9TB5Pyr1Dzhsj/5aQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'gvN7+feJ9DIIMXUudl6jdQFdPkDX7iTYc4uVI038Jn+PRXy4A/zgTpnXhp9TB5Pyr1Dzhsj/5aQ=',
   'x-amz-request-id': 'N3T5XEASVGEB4R5T',
   'date': 'Tue, 14 May 2024 15:22:43 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}